In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import json
import re
import pandas as pd
from agro_strategies import apply_agro_strategies

project = "import agribalyse 3.1"
bd.projects.set_current(project)

In [2]:
# Biosphere
bi.bw2setup()
bio = bd.Database("biosphere3")

Biosphere database already present!!! No setup is needed


In [ ]:
af_path = "Data/agribalyse_31_unit.csv"
af_name = "agribalyse 3.1"
af = bi.SimaProCSVImporter(
    filepath=af_path,
    name=af_name,
    delimiter=",",
)

af.apply_strategies()
af.match_database("biosphere3", fields=("name", "unit", "categories"))
af.statistics()

Extracted 17550 unallocated datasets in 80.53 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location


# Strategies and migrations

In [ ]:
migration_name = "agrifootprint-6-names"
bi.Migration(migration_name).write(
    json.load(open("Data/migrations/agrifootprint-6-economic.json")),
    "Change names of agrifootprint activities",
)
af.migrate(migration_name)

In [ ]:
af.apply_strategies()
af = apply_agro_strategies(af)
af.match_database("biosphere3", fields=("name", "unit", "categories"))
af.statistics()

# Agribalyse specific

In [ ]:
migration_name = "soil-categories"
bi.Migration(migration_name).write(
    json.load(open("Data/migrations/soil-categories.json")),
    "Change soil categories",
)
af.migrate(migration_name)
af.match_database("biosphere3", fields=("name", "unit", "categories"))
af.statistics()

In [ ]:
[act for act in bio if "Pirimicarb" in act['name']]

# Write biosphere database

Once you reach the point when you cannot link any more biosphere exchanges, and the remaining biosphere activities should just form a new database, then you need to write it as follows:

In [ ]:
ag_bio_name = "agrifootprint biosphere"
try:
    del bd.databases[ag_bio_name]
except:
    pass
#af.create_new_biosphere(ag_bio_name)
bd.Database(ag_bio_name).register()
af.add_unlinked_flows_to_biosphere_database(ag_bio_name)

# Write agrifootprint technosphere database

Finally, write technosphere database

In [ ]:
af.drop_unlinked(i_am_reckless=True)

In [ ]:
#af.add_unlinked_activities()
af.statistics()

In [ ]:
# After you deal with the substitution exchanges, you should be able to write the database:
ag_bio_name = "agrifootprint 6 economic"
try:
    del bd.databases[ag_bio_name]
except:
    pass
af.write_database()

# In case you want to share this bw project with others

In [ ]:
bi.backup_project_directory(project)

In [ ]:
# The created file is saved in your home directory. Other ppl can use it by running:
bi.restore_project_directory("give project path")
bd.projects.set_current(project)

# LCIA

In [ ]:
afdb = bd.Database('agrifootprint 6 economic')

In [ ]:
len(afdb)

In [ ]:
product_list = [act for act in afdb if 'Barley' in act['name']
                    and 'AR' in act['name']
                    and 'at farm' in act['name']
]
product_list

In [ ]:
method_list = []
for method in bd.methods:
    if 'EF v3.0' in method and "global warming potential (GWP100)" in method:
        method_list.append(method)


In [ ]:
for product in product_list:
    for lcia_method in method_list:
        lca = bc.LCA({product: 1}, method=lcia_method)
        lca.lci()
        lca.lcia()
        print(product, lcia_method[1],lca.score)